#CloakHat Patch Generation Pipeline

##1: Conda Environment Setup

`conda create -n cloakhat python=3.10 -y`
`conda activate cloakhat`

PyTorch with CUDA
`conda install pytorch torchvision pytorch-cuda=11.8 -c pytorch -c nvidia -y`

PyTorch3D for differentiable rendering
`pip install "git+https://github.com/facebookresearch/pytorch3d.git"`

Detection models
`pip install ultralytics`

Pip stuff
`pip install opencv-python-headless matplotlib tqdm tensorboard pyyaml trimesh`